# Saving and Loading ModelInducer() Configs

You can save your induction configurations and load them in another workspace to replicate a setup. 

In this notebook we will save and load an inducer config and replicate the setup in a different workspace.

# Check Environment Variables
Before installing Hybrid Intelligence in the notebook you need to set these Environment Variables externally as described in the User Guide https://docs.umnai.com/set-up-your-environment. 
This section checks that the environment variables have been set correctly and throws an error if not.

In [1]:
import os

umnai_env_vars = {
    'UMNAI_CLIENT_ID',
    'UMNAI_CLIENT_SECRET',
    'PIP_EXTRA_INDEX_URL',
}

if not umnai_env_vars.issubset(os.environ.keys()):
    raise ValueError(
        'UMNAI environment variables not set correctly. They need to be set before using the Umnai library.'
    )

# Install Hybrid Intelligence
Next we install the UMNAI Platform.

In [2]:
%pip install umnai-platform --quiet


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


# Set Workspace Paths
Now we will set the the experiment name and workspace paths.

In [3]:
EXP_NAME = 'saveloadConfig_adult_income'
EXP2_NAME = 'saveloadConfig_new_adult_income'

WS_PATH = 'resources/workspaces/'+EXP_NAME
WS2_PATH = 'resources/workspaces/'+EXP2_NAME

EXP_PATH = EXP_NAME
EXP2_PATH = EXP2_NAME

# Create or Open a Hybrid Intelligence Workspace
Workspaces are used by the Hybrid Intelligence framework to organize your data and models together in one place.

In [4]:
from umnai.workspaces.context import Workspace

# Open a workspace
ws = Workspace.open(
    path = WS_PATH,
    experiment = EXP_PATH
)

ws # Prints workspace details to confirm created/opened

WorkspaceContext(path=/opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/saveloadConfig_adult_income, experiment=saveloadConfig_adult_income, parallel_backend=loky, parallel_jobs=1)

# Create ModelInducer Config

In [5]:
from umnai.induction.inducer import ModelInducer

# Simple model inducer using fast execution parameters
model_inducer = ModelInducer(
    max_interactions=3,
    max_interaction_degree=2,
    max_polynomial_degree=2,
    trials=2,
    estimators=2,
    batch_size=512,
    iterations=2,
)

#### You can now use this Model Inducer to induce a Model.

# Save ModelInducer Config

In [6]:
# Save ModelInducer config
model_inducer.save_config(f'{WS_PATH}/{"resources/configs/inducer_config.json"}')

# Reuse the Config in a New Workspace
Next we will use the JSON file we just saved to recreate the induction setup in a new workspace.

# Create a New Hybrid Intelligence Workspace

In [7]:
# Close the open workspace as you can only have one workspace open at a time
ws.close()

# On a local machine environment:
ws2 = Workspace.open(
    path = WS2_PATH,
    experiment = EXP2_PATH
)

ws2 # Prints workspace details to confirm created/opened

WorkspaceContext(path=/opt/atlassian/pipelines/agent/build/demo-notebooks/resources/workspaces/saveloadConfig_new_adult_income, experiment=saveloadConfig_new_adult_income, parallel_backend=loky, parallel_jobs=1)

# Load ModelInducer Config
Load the config from the JSON file saved from another workspace.

In [8]:
# Load Model Inducer config from previous workspace
model_inducer_replicated = ModelInducer.load_config(file_path=f'{WS_PATH}/{"resources/configs/inducer_config.json"}')

Check parameters are same as those saved earlier:

In [9]:
# Confirm parameters as same:
print("max_interactions : ", model_inducer_replicated.max_interactions)
print("max_interaction_degree : ", model_inducer_replicated.max_interaction_degree)
print("max_polynomial_degree : ", model_inducer_replicated.max_polynomial_degree)
print("trials : ", model_inducer_replicated.trials)
print("estimators : ", model_inducer_replicated.estimators)
print("batch_size : ", model_inducer_replicated.batch_size)
print("iterations : ", model_inducer_replicated.iterations)

max_interactions :  3
max_interaction_degree :  2
max_polynomial_degree :  2
trials :  2
estimators :  2
batch_size :  512
iterations :  2


#### You can now use the replicated model inducer to induce a model.